__References__: 

    [1] - A departure-function approach to calculate thermodynamics properties of refrigerant-oil mixture, 
    Moisés A. Marcelino Neto, Jader R. Barbosa Jr., International Journal of Refrigeration, 36, (2013), 972-979 
    [2] - Applied Hydrocarbon Thermodynamics, Volume 1, Wayne C. Edmister, Byung Ik Lee, 2nd edition, 1984

In [1]:
import numpy as np
from scipy.constants import R as Rgi

In [2]:
class PengRobinson():
    '''Peng-Robinson's cubic EOS instantiation class for a N-component mixture. 
    
      
    Parameters
    ----------
    T : float or int
        fluid temperature (K)
    p : float or int
        fluid pressure (Pa)
    Tc : array-like
        critical temperature (K)
    pc : array-like
        critical pressure (Pa)
    xm : array-like
        molar fraction (-)
    M : array-like
        molar mass (kg/kmol)
    omega : array-like
        acentric factor
    
    Attributes: the name "Parameters", name used to represent the variables declared above, changes to "Attributes"
                when they goes into Class. Here, in this example, we have 10 attributes.
    ----------
    T : float or int
        fluid temperature (K)
    p : float or int
        fluid pressure (Pa)
    Tc : array-like
        critical temperature (K)
    pc : array-like
        critical pressure (Pa)
    xm : array-like
        molar fraction (-)
    M : array-like
        molar mass (kg/kmol)
    omega : array-like
        acentric factor (?)
    Tr : array-like
        reduced temperature (-)
    pr : array-like
        reduced pressure (-)
    N : int
        number of mixture components
    
    Methods
    -------
    *********TO DO************
    '''
    def __init__(self, T, p, Tc, pc, xm, M, omega,k):
        self.T = T
        self.p = p
        self.Tc = np.array(Tc)
        self.pc = np.array(pc)
        self.xm = np.array(xm)
        self.M = np.array(M)
        self.omega = np.array(omega)
        self.k = np.array(k)
        self.Tr = T / self.Tc
        self.pr = p / self.pc
        self.N = self.Tc.shape[0]  #contou nº linhas na array; se fosse ...shape[1] seria nº colunas
    
    
    def kappa_pr(self):
        omega = self.omega
        return 0.37464 + 1.54226 * omega - 0.26993 * omega ** 2 
          
    def alpha_pr(self):
        kappa = self.kappa_pr()
        Tr = self.Tr
        return (1 + kappa * (1 - Tr ** 0.5)) ** 2
    
    def ai_pr(self):
        alpha = self.alpha_pr()
        Tc = self.Tc
        pc = self.pc
        return 0.45723553 * alpha * ( Rgi ** 2 * Tc ** 2 ) / pc
    
    def a_pr(self):
        '''Lacks documentation,
        sofisticated methods'''
        xm = self.xm
        ai = self.ai_pr()
        k =  self.k               
        aij = np.einsum('i,j->ij', ai, ai)** 0.5 * (1 - k)
        return np.sum(np.einsum('i,j->ij', xm, xm)* aij)
    
    def A_pr(self):
        a = self.a_pr()
        p = self.p
        T = self.T
        return (a * p) / (Rgi **2 * T **2)
           
    def bi_pr(self):
        Tc = self.Tc
        pc = self.pc
        return 0.07779607 * ( Rgi * Tc ) / pc
    
    def b_pr(self):
        xm = self.xm
        bi = self.bi_pr()
        return np.dot(xm, bi)
    
    def B_pr(self):
        b = self.b_pr()
        p = self.p
        T = self.T
        return (b * p) / (Rgi * T)
    
    def Z_coefs(self):
        A = self.A_pr()
        B = self.B_pr()
        return(1, -( 1 - B ),
               (A - 3 * B ** 2 - 2 * B),
               -( A * B - B ** 2 - B ** 3 ))   
    


### A class TakingCorrectRoots "erda" os atributos da PengRobinson class + "phase". 

In [3]:
class TakingCorrectRoots(PengRobinson):
    
    def __init__(self, T, p, Tc, pc, xm, M, omega,k,phase):
        PengRobinson.__init__(self, T, p, Tc, pc, xm, M, omega,k)
        self.phase = phase
    
        
    def __call__(self):
        coefs = self.Z_coefs() #olha como chamar um método de outra classe que tem vários outros métodos
        roots = np.roots(coefs)
        real_roots = roots[roots.imag == 0]
        physical_roots = real_roots[real_roots >= 0]
        phase = self.phase
        phase = phase.upper()
        if phase == 'L':
            return physical_roots.min()
        elif phase == 'V':
            return physical_roots.max()
        else:
            return ValueError('phase must be either L or V')
        
        
    

In [4]:
class Phi(TakingCorrectRoots):
    def __init__(self,T, p, Tc, pc, xm, M, omega,k,phase):
        TakingCorrectRoots.__init__(self, T, p, Tc, pc, xm, M, omega,k,phase)
        self.Z = TakingCorrectRoots(T, p, Tc, pc, xm, M, omega,k,phase)
    
        
    def __call__(self): 
        ai = self.ai_pr()
        bi = self.bi_pr()
        a = self.a_pr()
        b = self.b_pr()
        A = self.A_pr()
        B = self.B_pr()
        k = np.array([0,0,0,0])
        xm = self.xm
        sub_1 = xm * np.sqrt(ai)
        sub_2 = 1 - k
        soma = sub_1[:, None] * sub_2
        soma2 = soma.ravel()[::self.N].reshape((self.N,self.N)).sum(axis=0)
        Aiprime = (1 / a) * 2 * np.sqrt(ai) * soma2
        Biprime = bi / b
        Z = self.Z()
        term1 = np.log(Z-B)
        term2 = (Z-1) * Biprime
        term3 = A / (2 ** 1.5 * B)
        term4 = Aiprime - Biprime
        term5 = np.log( (Z + (2**0.5 + 1) * B) / (Z - (2**0.5 -1) * B) )
        term = -term1 + term2 - term3*term5*term4
        phi = np.exp(term)
        return phi        
    
    

__Comentário__

>>> A class Phi erda da class TakingCorrectRoots. Observe como foi diferente chamar o método de uma class que só tem um método, o método \__call__. Veja as diferenças para chamar e posições:

(1ª) coefs = self.Z_coefs() que foi o método chamado da Class PengRobinson (que tem vários métodos)

(2ª) self.Z = TakingCorrectRoots(T, p, Tc, pc, xm, M, omega,k,phase) que foi o método \__call__ da Class TakingCorrectRoots

__Input data__

In [11]:
T = 100
p = 0.4119E+6
Tc = [126.1, 190.6]
pc = [3.394E+6, 4.604E+6]
M = [28, 16]
omega = [0.04, 0.011]
xm = [0.958, 1-0.958]
k = [[0.0,0.0],[0.0,0.0]]  #binary interaction parameter; I have set zero initially


__Instancing the Class__

In [12]:
pr = PengRobinson(T,p,Tc,pc,xm,M,omega,k)
Z_v = TakingCorrectRoots(T,p,Tc,pc,xm,M,omega,k,'V')
Z_l = TakingCorrectRoots(T,p,Tc,pc,xm,M,omega,k,'V')
phi_v = Phi(T,p,Tc,pc,xm,M,omega,k,'V')
phi_l = Phi(T,p,Tc,pc,xm,M,omega,k,'L')

__Testing__

In [13]:
print(pr.Tr, pr.pr)

[ 0.79302141  0.52465897] [ 0.12136123  0.08946568]


In [14]:
fatorAcentrico = pr.omega
print(fatorAcentrico)

[ 0.04   0.011]


In [15]:
print(Z_v(), Z_l())

0.905911525621 0.905911525621


In [16]:
print(phi_v(), phi_l())

[ 0.91615978  0.84729268] [ 1.57283683  0.13703829]
